**Setup in Google Colab**

In [ ]:
!pip install langchain langchain-community
!pip install faiss-cpu
!pip install pypdf python-docx
!pip install sentence-transformers
!pip install transformers


**Upload File in Colab**

In [ ]:
from ipywidgets import FileUpload
from IPython.display import display

uploaded = FileUpload(accept='.pdf,.docx,.txt', multiple=False)
display(uploaded)

def handle_upload(change):
    if uploaded.value:
        # uploaded.value is a tuple → take first element
        file_info = list(uploaded.value.values())[0] if isinstance(uploaded.value, dict) else list(uploaded.value)[0]
        
        # Get filename (some versions use 'name', older ones used metadata.name)
        file_name = file_info.get('name') or file_info.get('metadata', {}).get('name')
        
        # Save file locally
        with open(file_name, "wb") as f:
            f.write(file_info['content'])
        
        print("Uploaded:", file_name)

        # Make it available as file_path (like Colab)
        global file_path
        file_path = file_name

uploaded.observe(handle_upload, names='value')


**Load & Split Document**

In [ ]:
from langchain_community.document_loaders import PyPDFLoader, TextLoader, Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Pick loader based on file type
if file_path.endswith(".pdf"):
    loader = PyPDFLoader(file_path)
elif file_path.endswith(".docx"):
    loader = Docx2txtLoader(file_path)
else:
    loader = TextLoader(file_path)

docs = loader.load()

# Split into smaller chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
documents = splitter.split_documents(docs)

print(f"Total Chunks: {len(documents)}")


**Create Embeddings & Vector Store**

In [38]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Small + Fast embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = FAISS.from_documents(documents, embeddings)


**Load an Open-Source LLM**

In [39]:
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline

# Load small model
flan_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",   # change to flan-t5-small for even faster
    max_length=512
)

llm = HuggingFacePipeline(pipeline=flan_pipeline)


Device set to use cpu


**Retrieval Q&A**

In [40]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    chain_type="stuff"
)

# Test
query = "Give me a short summary of the document"
print(qa.run(query))


Project Neptune: Phase One


**Chat Loop**

In [35]:
while True:
    q = input("Ask a question (or 'exit'): ")
    if q.lower() == "exit":
        break
    print("Answer:", qa.run(q))


Answer: Alice Johnson
Answer: September 1, 2025
Answer: design the user interface for the feedback system
Answer: Develop the core data processing module. - Integrate with the existing analytics dashboard. - Create a user feedback collection system.
